In [1]:
import cv2
import os
import numpy as np
import math
from google.colab.patches import cv2_imshow
from IPython.core.display import display, HTML
import json
import traceback

In [2]:
orb = cv2.ORB_create(nfeatures=1500,scoreType=cv2.ORB_FAST_SCORE)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

In [3]:
with open('/content/drive/MyDrive/IsaacItemID/ItemDictionary.json', 'r') as fp:
    itemDict = json.load(fp)

In [4]:
with open('/content/drive/MyDrive/IsaacItemID/ItemFeatures.json', 'r') as fp:
    imageDict = json.load(fp)

In [6]:
def getSimilarImages(imageToFindPath,imageDict,topN_features=10):

    imageToFind = cv2.imread(imageToFindPath,cv2.IMREAD_GRAYSCALE)
    kp, des = orb.detectAndCompute(imageToFind,None)

    similarityDict = {}

    for item in imageDict:
        if imageDict[item]['des']:
            itemDesc = np.array(imageDict[item]['des'],dtype='uint8')
            # print(type(itemDesc),type(des))
            matches = bf.match(itemDesc, des)

            topN = topN_features
            matches = sorted(matches, key = lambda x:x.distance)
      
            matches = matches[:topN]

            list_kp = []
            # For each match...
            for mat in matches:
                # Get the matching keypoints for the image
                img_idx = mat.trainIdx
                # Get the coordinates
                (x, y) = kp[img_idx].pt
                # Append to each list
                list_kp.append((x, y))

            # compute the average x,y point for the top n distances
            totalXPixel = 0
            totalYPixel = 0

            for point in list_kp:
                totalXPixel += point[0]
                totalYPixel += point[1]

            avgXPixelDistance = totalXPixel / len(list_kp)
            avgYPixelDistance = totalYPixel / len(list_kp)

            distances = []

            for p in list_kp:
                # compute the average distance from a point to the avg point
                a2 = (p[0] - avgXPixelDistance)**2
                b2 = (p[1] - avgYPixelDistance)**2
                distance = math.sqrt(a2+b2)
                distances.append(distance)

            mean = sum(distances) / len(distances)
            # print('Mean',mean)
            res = sum((k - mean) ** 2 for k in distances) / len(distances)
            # print('Variance',res)

            similarityDict[item] = {'Mean':mean,'Variance':res,'Matches':matches}
    return similarityDict


In [7]:
def displayImageComparison(items,image):
    # input image
    image = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
    for item in items:
        path = imageDict[item]['imgPath']
        # stored icon
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(128,128))

        # icon data
        kp1, des1 = orb.detectAndCompute(img,None)
        kp2, des2 = orb.detectAndCompute(image,None)

        matches = bf.match(des1, des2)
        topN = 10
        matches = sorted(matches, key = lambda x:x.distance)
        matches = matches[:topN]

        matching_result = cv2.drawMatches(img,kp1,image,kp2,matches,None)
        cv2_imshow(matching_result)

In [8]:
def findItem(path,topN=5,topN_features=10,show_found_images=True,display_comparison_data=False,display_stats=False):
    similarImagesDictionary = getSimilarImages(path,imageDict,topN_features=topN_features)
    itemOrder = sorted(similarImagesDictionary,key=lambda x:similarImagesDictionary[x]['Variance'])

    for itemName in itemOrder[:topN]:
        imgPath = itemDict[itemName]['ImagePath']
        image = cv2.imread(imgPath,cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        image = cv2.resize(image,(128,128))
        if show_found_images:
            cv2_imshow(image)
        print('Item:',itemName)
        print('Effects:')
        print(display(HTML(itemDict[itemName]['Effects'])))
  
    if display_comparison_data:
        displayImageComparison(itemOrder[:topN],path)